In [3]:
import os
from lips import get_root_path

In [4]:
LIPS_PATH = get_root_path()
DIRECTORY_NAME = 'Dataset'
BENCHMARK_NAME = "Case1"
LOG_PATH = LIPS_PATH + "lips_logs.log"
BENCH_CONFIG_PATH = os.path.join("airfoilConfigurations","benchmarks","confAirfoil.ini") #Configuration file related to the benchmark
SIM_CONFIG_PATH = os.path.join("airfoilConfigurations","simulators","torch_fc.ini") #Configuration file re

In [5]:
from lips.dataset.airfransDataSet import download_data
if not os.path.isdir(DIRECTORY_NAME):
    download_data(root_path=".", directory_name=DIRECTORY_NAME)

In [6]:
from lips.benchmark.airfransBenchmark import AirfRANSBenchmark

benchmark=AirfRANSBenchmark(benchmark_path = DIRECTORY_NAME,
                            config_path = BENCH_CONFIG_PATH,
                            benchmark_name = BENCHMARK_NAME,
                            log_path=LOG_PATH)
                            
benchmark.load(path=DIRECTORY_NAME)

Loading dataset (task: reynolds, split: test): 100%|██████████| 496/496 [02:25<00:00,  3.40it/s]


In [7]:
benchmark.train_dataset.data["x-position"]

array([ 4.21687031,  4.21688938,  3.99183941, ..., -2.13636851,
       -2.14823389, -2.16010475])

In [12]:
import torch
from torch_geometric.transforms import BaseTransform
from torch_geometric.data import Data
from scipy.spatial import Delaunay
import numpy as np

class DelaunayTransform(BaseTransform):
    def __init__(self, dim=2):
        """
        Initialize the DelaunayTransform.

        Args:
            dim (int): Dimensionality of the points (default is 2 for 2D).
        """
        self.dim = dim

    def __call__(self, data: Data) -> Data:
        """
        Apply Delaunay triangulation to the node coordinates to construct edge_index.

        Args:
            data (Data): PyTorch Geometric Data object with 'x' attribute.

        Returns:
            Data: Updated Data object with 'edge_index' constructed via Delaunay triangulation.
        """
        # Convert node features to NumPy arrays
        points = data.pos.cpu().numpy()
        surf = data.surf.cpu().numpy()

        # Perform Delaunay triangulation
        tri = Delaunay(points)

        # Check if all nodes in simplex are on the surface
        simplices_surf = surf[tri.simplices]
        all_on_surf = simplices_surf.all(axis=1)
        simplices_to_use = ~all_on_surf
        valid_simplices = tri.simplices[simplices_to_use]

        # Extract edges from valid simplices
        edges = np.concatenate([
            valid_simplices[:, [0, 1]],
            valid_simplices[:, [1, 2]],
            valid_simplices[:, [2, 0]],
            valid_simplices[:, [1, 0]],
            valid_simplices[:, [2, 1]],
            valid_simplices[:, [0, 2]]
        ], axis=0)

        edge_index = np.unique(edges, axis=0)

        # Convert edge_index to torch tensor
        edge_index = torch.tensor(edge_index, dtype=torch.long, device=data.pos.device)

        # Update the Data object
        data.edge_index = edge_index.t()
        return data


In [13]:
import numpy as np
from torch_geometric.data import Data, DataLoader

In [16]:
def process_dataset(dataset, training: bool) -> DataLoader:
    coord_x=dataset.data['x-position']
    coord_y=dataset.data['y-position']
    surf_bool=dataset.extra_data['surface']

    position = np.stack([coord_x,coord_y],axis=1)

    nodes_features, node_labels = dataset.extract_data()

    torchDataset=[]
    nb_nodes_in_simulations = dataset.get_simulations_sizes()
    start_index = 0
    for nb_nodes_in_simulation in nb_nodes_in_simulations:
        end_index = start_index+nb_nodes_in_simulation
        simulation_positions = torch.tensor(position[start_index:end_index,:], dtype = torch.float) 
        simulation_features = torch.tensor(nodes_features[start_index:end_index,:], dtype = torch.float) 
        simulation_labels = torch.tensor(node_labels[start_index:end_index,:], dtype = torch.float) 
        simulation_surface = torch.tensor(surf_bool[start_index:end_index])

        sampleData=Data(pos=simulation_positions,
                        x=simulation_features, 
                        y=simulation_labels,
                        surf = simulation_surface.bool()) 
        sampleData = DelaunayTransform()(sampleData)
        
        # Nombre de nœuds dans le graphe (le plus grand index + 1)
        num_nodes = sampleData.edge_index.max().item() + 1

        # Calcul du degré de chaque nœud
        degrees = torch.bincount(sampleData.edge_index[0], minlength=num_nodes)

        # Calcul du degré moyen
        average_degree = degrees.float().quantile(0.5).item()
        print(f'Median degree: {average_degree}')

        torchDataset.append(sampleData)
        start_index += nb_nodes_in_simulation
    return DataLoader(dataset=torchDataset,batch_size=1)

data_loader = process_dataset(benchmark.train_dataset, True)

KeyboardInterrupt: 

## Example point cloud to graph

In [9]:
from torch_geometric.transforms import SamplePoints, KNNGraph
import torch_geometric.transforms as T
from torch_geometric.datasets import GeometricShapes
import networkx as nx
from torch_geometric.utils import to_networkx
from matplotlib import pyplot as plt
# Generate a dataset of 2d points

#transform = DelaunayTransform(3)
#print(dataset[1].pos)
#data = transform(dataset[1])
#g = to_networkx(ata, to_undirected=True)
#nx.draw(g, node_size=10)

## Simultion point cloud to graph

In [10]:
data = None
g = None
for data in data_loader:
    data = Data(pos=data.pos, x=data.x, y=data.y)

    # Apply the KNNGraph transformation
    transform = DelaunayTransform()
    data = transform(data)
    
    data = Data(pos=data.pos, x=data.x, y=data.y, edge_index=data.edge_index, edge_attr=data.edge_attr)
    g = to_networkx(data, to_undirected=False)
    nx.write_gexf(g, "test.gexf")
    break

import pandas as pd

# Supposons que 'data' soit ton graphe PyTorch Geometric
node_ids = range(data.num_nodes)
df_nodes = pd.DataFrame(node_ids, columns=['Id'])

# Si les caractéristiques des nœuds sont présentes, on peut les ajouter
if data.x is not None:
    df_nodes['Features'] = data.x.tolist()

# Ajouter les positions des nœuds si elles existent
if data.pos is not None:
    df_nodes['X'] = data.pos[:, 0].tolist()
    df_nodes['Y'] = data.pos[:, 1].tolist()
    if data.pos.size(1) > 2:  # Si les positions sont en 3D
        df_nodes['Z'] = data.pos[:, 2].tolist()

# Sauvegarder en CSV pour importation dans Gephi
df_nodes.to_csv("nodes_with_positions.csv", index=False)

# Exportation des arêtes
edges = data.edge_index.t().numpy()  # Convertir en tableau numpy
df_edges = pd.DataFrame(edges, columns=['Source', 'Target'])

# Sauvegarder en CSV pour importation dans Gephi
df_edges.to_csv("edges.csv", index=False)


In [11]:
# Use the original positions from data.pos for node placement
# pos = {i: data.pos[i].numpy() for i in range(data.pos.shape[0])}
# plt.figure(figsize=(12, 12)) 
# # Draw only the nodes (without edges)
# nx.draw_networkx_nodes(g, pos, node_size=5, node_color='r')
# print(type(g))
# nx.draw_networkx_edges(g, pos)
# plt.savefig("graph_output.svg", format="svg")
#plt.scatter(data.pos[:, 0], data.pos[:, 1], c=data.y[:, 0], cmap='coolwarm', s=20)